In [1]:
import logging

import numpy as np
import optimistix as optx
from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
    SolverParameters,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm

from scipy.interpolate import interp1d
import os

from matplotlib.ticker import ScalarFormatter

formatter = ScalarFormatter(useMathText=False)
formatter.set_scientific(False)
formatter.set_useOffset(False)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

logger = debug_logger() #you can dump it in file 
#logger.setLevel(logging.INFO) #you could set it to error

logger.setLevel(logging.DEBUG)  
##rebase for atmodeller git pull, git re base (you might also have to git pull when paralell)

Atmodeller initialized with double precision (float64)


In [2]:
#Setting up the different layers for Uranus 

#Overall structure 
R_earth = 6.371e6
R_uranus = 2.5388e7 #meters
M_earth = 5.972e24 #kg
M_uranus = 14.536*M_earth

#Boundary inner and outer envelope
boundary_mass = 0.913*M_uranus 
boundary_radius = 0.772*R_uranus
boundary_temperature = 2338.8
boundary_core_mass_fraction = 0
boundary_mantle_melt_fraction = 0

#Core 
core_mass = 0.61*M_earth 
core_mass_fraction_U = 0
core_mantle_melt_fraction = 1.0
core_radius = 0.7*R_earth 
core_temperature = 6000

Z_1 = 0.17
Z_2 = 0.915
Z_solar = 0.0142
RANDOM_SEED = 0

np.random.seed(RANDOM_SEED)

In [3]:
H_masspercent_lodders09: float = 73.9
He_masspercent_lodders09: float = 24.69
C_masspercent_lodders09: float = 0.22
N_masspercent_lodders09: float = 0.07
O_masspercent_lodders09: float = 0.63
Si_masspercent_lodders09: float = 0.07
Mg_masspercent_lodders09: float = 0.06

H_logN: float = 12
He_logN: float = 10.93
C_logN: float = 8.39
N_logN: float = 7.86
O_logN: float = 8.73
Si_logN: float = 7.53
Mg_logN: float = 7.54 

# Boundary inner envelope and core

In [4]:
#SiO2 for primitive composition of Earth
SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme141: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999

MgO_mantlemasspercent_palme14: float = 36.77
Mg_massfraction_palme14: float = round(
    MgO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (24.305 + 15.999)
    * 24.305,
    3,
)

FeO_mantlemasspercent_palme14: float = 8.10
Fe_massfraction_palme14: float = round(
    FeO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (55.845 + 15.999)
    * 55.845,
    3,
)

O_massfraction_palme142: float = Mg_massfraction_palme14 / 24.305 * 15.999
O_mass_fraction_palme143: float = Fe_massfraction_palme14 / 55.845 * 15.999
O_massfraction_palme14 = O_massfraction_palme141 + O_massfraction_palme142
print(O_massfraction_palme14)
print(Si_massfraction_palme14)

0.394198674005231
0.216


In [5]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = Species.create_condensed("O2Si", state="l")
O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
#
CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas(
    "N2", solubility=sol_models["N2_basalt_libourel03"], activity=eos_models["N2_cs_saxena87"]
)

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = Species.create_gas("Mg")
#Mg_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = Species.create_condensed("MgO", state="l")
MgO_cr = Species.create_condensed("MgO", state="cr")
MgH_g = Species.create_gas("MgH")
MgOH_g = Species.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection(
    (H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l)
     #)
)
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection(
    (H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l)
     #, )
)
species_HHeCNOSi_magma_sol_real = SpeciesCollection(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l
    )
)

## define mass by metallicity

In [7]:
M_inner_envelope = boundary_mass - core_mass
M_outer_envelope = M_uranus - boundary_mass
M_heavy = Z_2*M_inner_envelope + Z_1*M_outer_envelope
He_mass = He_masspercent_lodders09 * (M_uranus - M_heavy- core_mass) / 100
H_mass = H_masspercent_lodders09 * (M_uranus - core_mass) / 100
print(M_inner_envelope)
print(M_outer_envelope)
print(M_heavy)
print(He_mass)
print(H_mass) 
print(M_heavy * O_masspercent_lodders09 / 100)

7.5613689696e+25
7.552382304000005e+24
7.047043106352e+25
3.134553747216913e+24
6.145972720800001e+25
4.43963715700176e+23


In [19]:
#H_mass_final = H_masspercent_lodders09 / 100 * M_heavy + H_mass
#He_mass_final =  He_mass
#print(H_mass_final)
hmps = np.array([0.15])
H_mass = hmps*M_uranus

In [20]:
si_kg_magma: float = Si_massfraction_palme14 * core_mass
mg_kg_magma: float = Mg_massfraction_palme14 * core_mass
o_kg_magma: float = O_massfraction_palme14 * core_mass

# Lodders et al. (2009) Springer book chapter Table 8
si_kgs_solar = M_heavy * Si_masspercent_lodders09 / 100 
mg_kgs_solar = M_heavy * Mg_masspercent_lodders09 / (100* (1-He_masspercent_lodders09-H_masspercent_lodders09))
o_kgs_solar = M_heavy * O_masspercent_lodders09 / 100
c_kgs_solar = M_heavy * C_masspercent_lodders09 / (100* (1-He_masspercent_lodders09-H_masspercent_lodders09))
n_kgs_solar = M_heavy * N_masspercent_lodders09 / (100* (1-He_masspercent_lodders09-H_masspercent_lodders09))
he_kgs_solar = M_heavy * He_masspercent_lodders09 / 100
print(o_kgs_solar)
print(H_mass)
print(He_mass)
print(c_kgs_solar)
print(n_kgs_solar)
print(core_mantle_melt_fraction * si_kg_magma + si_kgs_solar)
print(core_mantle_melt_fraction * o_kg_magma + o_kgs_solar)
mass_constraints = {
    "H": H_mass,
#    "He": He_mass,
#    "C": c_kgs_solar,
#    "N": n_kgs_solar,
    "Si":  core_mantle_melt_fraction * si_kg_magma + si_kgs_solar,
    "O": core_mantle_melt_fraction * o_kg_magma + o_kgs_solar ,
#    "Mg": metall * mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma,
        }
#solver = optx.LevenbergMarquardt, 
solver_parameters = SolverParameters(multistart= 50)
Uranus = Planet(
    surface_temperature=core_temperature,
    planet_mass=core_mass,
    mantle_melt_fraction=core_mantle_melt_fraction,
    core_mass_fraction = core_mass_fraction_U,
    surface_radius=core_radius,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

#last minute solution, reverting to other solver 
model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_density=initial_log_number_density
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Core_real_sol_new_test")

[15:41:28 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility')
[15:41:28 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 1: '3.0 H2_g + 1.0 OSi_g = 1.0 H2O_g + 1.0 H4Si_g',
 2: '1.0 H2O_g + 1.0 OSi_g = 1.0 H2_g + 1.0 O2Si_l'}
[15:41:28 - atmodeller.classes             - INFO     ] - Attempting to solve 1 model(s)
[15:41:28 - atmodeller.classes             - INFO     ] - Solve complete: 1 (100.00%) successful model(s)
[15:41:28 - atmodeller.classes             - INFO     ] - Solver steps (max) = 38
[15:41:28 - atmodeller.output              - DEBUG    ] - Creating Output


4.43963715700176e+23
[1.30213488e+25]
3.134553747216913e+24
-1.5886356013909622e+21
-5.0547496407894255e+20
8.362000217444639e+23
1.879997949207312e+24


[15:41:28 - atmodeller.output              - INFO     ] - Writing output to excel
[15:41:28 - atmodeller.output              - INFO     ] - Computing to_dataframes output
[15:41:28 - atmodeller.output              - INFO     ] - Computing asdict output
[15:41:28 - atmodeller.output              - DEBUG    ] - Found O2_g so back-computing log10 shift for fO2
[15:41:29 - atmodeller.output              - INFO     ] - Output written to HHeCNOSi_magma_nosol_ideal.xlsx
[15:41:29 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility')
[15:41:29 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 1: '3.0 H2_g + 1.0 OSi_g = 1.0 H2O_g + 1.0 H4Si_g',
 2: '1.0 H2O_g + 1.0 OSi_g = 1.0 H2_g + 1.0 O2Si_l'}
[15:41:29 - atmodeller.

## define mass by ice to rock ratio

In [56]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = Species.create_condensed("O2Si", state="l")
O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
#
CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas(
    "N2", solubility=sol_models["N2_basalt_libourel03"], activity=eos_models["N2_cs_saxena87"]
)

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = Species.create_gas("Mg")
#Mg_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = Species.create_condensed("MgO", state="l")
MgO_cr = Species.create_condensed("MgO", state="cr")
MgH_g = Species.create_gas("MgH")
MgOH_g = Species.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection(
    (H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l)
     #)
)
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection(
    (H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l)
     #, )
)
species_HHeCNOSi_magma_sol_real = SpeciesCollection(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l,
        Mg_g, 
        MgO_l,
        MgH_g,
        MgOH_g
    )
)

In [18]:
ice_rock_ratio = 19
M_rock = M_heavy / (1+ice_rock_ratio)
M_ice = M_heavy - M_rock
print(M_ice)
print(M_rock)

6.6946909510344e+25
3.523521553176e+24


In [64]:
# ZCH4: ZNH3: ZH2O = 0.31 : 0.08 : 0.61
Z_CH4 = 0.31
Z_NH3 = 0.08
Z_H2O = 0.61

M_CH4 = Z_CH4 * M_ice
M_NH3 = Z_NH3 * M_ice
M_H2O = Z_H2O * M_ice

o_kgs_ice = M_H2O *  15.999 / ( 15.999 + 2 * 1.00794)
h_kgs_ice = M_H2O *  2 * 1.00794 / ( 15.999 + 2 * 1.00794) + M_NH3 * 3 * 1.00794 / ( 14.0067 + 3 * 1.00794) + M_CH4 * 4 * 1.00794 / ( 12.0107 + 4 * 1.00794)
n_kgs_ice = M_NH3 * 14.0067 / ( 14.0067 + 3 * 1.00794)
c_kgs_ice = M_CH4 * 12.0107 / ( 12.0107 + 4 * 1.00794)

h_kgs_solar = (M_uranus - boundary_mass) * H_masspercent_lodders09 / 100
he_kgs_solar = (M_uranus - boundary_mass) * He_masspercent_lodders09 / 100
si_kgs_solar = (M_uranus - boundary_mass) * Si_masspercent_lodders09 / 100
o_kgs_solar = (M_uranus - boundary_mass) * O_masspercent_lodders09 / 100
c_kgs_solar = (M_uranus - boundary_mass) * C_masspercent_lodders09 / 100
n_kgs_solar = (M_uranus - boundary_mass) * N_masspercent_lodders09 / 100
mg_kgs_solar = (M_uranus - boundary_mass) * Mg_masspercent_lodders09 / 100

print(h_kgs_ice + h_kgs_solar)
print(he_kgs_solar)
print(c_kgs_ice + c_kgs_solar)
print(n_kgs_ice +  n_kgs_solar)
print(core_mantle_melt_fraction * si_kg_magma + si_kgs_solar)
print(core_mantle_melt_fraction * o_kg_magma + o_kgs_ice + o_kgs_solar)
print(mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma)

1.6317643452611972e+25
1.8646831908576014e+24
1.5554417191475754e+25
4.410109432571405e+24
7.921573876127999e+23
3.721191900385617e+25
8.241884293824e+23


In [66]:
mass_constraints = {
    "H": h_kgs_ice + h_kgs_solar,
#    "He": he_kgs_solar,
#    "C": c_kgs_ice + c_kgs_solar,
#    "N": n_kgs_ice +  n_kgs_solar,
    "Si":  core_mantle_melt_fraction * si_kg_magma + si_kgs_solar,
    "O": core_mantle_melt_fraction * o_kg_magma + o_kgs_ice + o_kgs_solar ,
    "Mg": mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma,
        }
#solver = optx.LevenbergMarquardt, 
solver_parameters = SolverParameters(multistart= 50)
Uranus = Planet(
    surface_temperature=4600,
    planet_mass=core_mass,
    mantle_melt_fraction=core_mantle_melt_fraction,
    core_mass_fraction = core_mass_fraction_U,
    surface_radius=core_radius,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

#last minute solution, reverting to other solver 
model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_density=initial_log_number_density
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Core_real_sol_new_test")

[00:39:14 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility')
[00:39:15 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 1: '3.0 H2_g + 1.0 OSi_g = 1.0 H2O_g + 1.0 H4Si_g',
 2: '1.0 H2O_g + 1.0 OSi_g = 1.0 H2_g + 1.0 O2Si_l'}
[00:39:15 - atmodeller.classes             - INFO     ] - Attempting to solve 1 model(s)
[00:39:15 - atmodeller.classes             - INFO     ] - Solve complete: 1 (100.00%) successful model(s)
[00:39:15 - atmodeller.classes             - INFO     ] - Solver steps (max) = 39
[00:39:15 - atmodeller.output              - DEBUG    ] - Creating Output
[00:39:16 - atmodeller.output              - INFO     ] - Writing output to excel
[00:39:16 - atmodeller.output              - INFO     ] - Computing to_data